In [106]:
import torch
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import librosa
import pandas as pd
import numpy as np

In [109]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53")

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

/Users/jen/mambaforge/envs/torch-gpu/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [112]:
def preprocess_file(file_path):
    # Load and preprocess the audio file
    y, sr = librosa.load(file_path, mono=True)  # Read the .wav file 
    y = librosa.resample(y, orig_sr=sr, target_sr=16000)

    input_values = feature_extractor(y, sampling_rate=16000, return_tensors="pt").input_values  

    # Extract feature vector
    with torch.no_grad():
        logits = model(input_values).extract_features
    feature_vector = logits[0]  # Shape: (audio_length_in_frames, 768)

    return feature_vector.numpy() 

In [104]:
train_df_cc = pd.read_csv('datasets/addresso/train/cc_meta_data.csv')
train_df_cd = pd.read_csv('datasets/addresso/train/cd_meta_data.csv')
train_df_cc['path'] = 'datasets/addresso/train/concatenated/cc/' + train_df_cc['ID'] + '.wav'
train_df_cd['path'] = 'datasets/addresso/train/concatenated/cd/' + train_df_cd['ID'] + '.wav'
train_df_cc['class'] = 0
train_df_cd['class'] = 1

train_df = pd.concat([train_df_cc, train_df_cd])
train_df['features'] = train_df['path'].apply(preprocess_file)
train_df['length'] = train_df['features'].apply(lambda x: len(x))

train_df.head()

KeyboardInterrupt: 

In [85]:
X = np.stack(train_df['features'])
y = train_df['class']

ValueError: all input arrays must have the same shape

In [75]:
len(X)

108

In [113]:
x = preprocess_file("datasets/addresso/train/concatenated/cc/S001.wav")

print(x.shape)

(2581, 512)


In [52]:
from sklearn.svm import SVC

In [53]:
clf = SVC(kernel='linear')

In [58]:
clf.fit(X, y)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (108,) + inhomogeneous part.